In [19]:
"""
Exercise 02: State Design Challenge - Starter Code
===================================================
Design and implement state with reducers for a support ticket workflow.

LEARNING GOALS:
- Use Annotated with custom reducers
- Design state for multi-step workflows
- Understand when to use reducers vs regular fields
"""

import os
from dotenv import load_dotenv
from typing import TypedDict, Literal, Annotated

from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model

load_dotenv()


# =============================================================================
# TODO 1: Implement a Custom Reducer
# =============================================================================
# A reducer function receives (existing_value, new_value) and returns merged.
# Use for fields that should ACCUMULATE instead of REPLACE.
#
# Create log_reducer that:
# - Takes existing list and new list
# - Returns combined list (existing + new)
#
# EXPERIMENT: Add timestamps to log entries
# EXPERIMENT: Limit log to last N entries
# =============================================================================

def log_reducer(existing: list[str], new: list[str]) -> list[str]:
    """Reducer that appends new log entries to existing log."""
    if existing is None:
        existing = []
    if new is None:
        new = []
    return existing + new

In [20]:

# =============================================================================
# TODO 2: Design the State with Reducers
# =============================================================================
# Create a TypedDict with:
# - ticket_content: str
# - customer_id: str
# - category: str (billing, technical, general)
# - urgency: int (1-5)
# - response_draft: str
# - processing_log: list[str] with your reducer
#
# Use Annotated[type, reducer] for fields that should accumulate.
#
# EXPERIMENT: Add a "tags" field that accumulates
# =============================================================================

class TicketState(TypedDict):
    """State for customer support ticket processing."""
    ticket_content: str
    customer_id: str
    category: Literal["billing", "technical", "general"]
    urgency: int
    response_draft: str
    processing_log: Annotated[list[str], log_reducer]
    tags: Annotated[list[str], log_reducer]


In [21]:

# =============================================================================
# TODO 3: Implement Processing Nodes
# =============================================================================
# Each node should:
# - Process the ticket
# - Add an entry to processing_log (it will accumulate!)
# - Return only the fields being updated
#
# EXPERIMENT: What happens if you return processing_log without a list?
# =============================================================================

def categorize_ticket(state: TicketState) -> dict:
    """Categorize into billing, technical, or general."""
    model = init_chat_model("gpt-4o-mini", model_provider="openai")
    
    prompt = f"""Categorize the following support ticket as one of: billing, technical, or general.

Ticket content:
{state['ticket_content']}

Respond with only one word: billing, technical, or general."""
    
    response = model.invoke(prompt)
    category = response.content.strip().lower()
    
    if category not in ["billing", "technical", "general"]:
        category = "general"
    
    log_entry = f"Categorized ticket as: {category}"
    
    return {
        "category": category,
        "processing_log": [log_entry]
    }


def assess_urgency(state: TicketState) -> dict:
    """Assess urgency from 1 (low) to 5 (critical)."""
    model = init_chat_model("gpt-4o-mini", model_provider="openai")
    
    prompt = f"""Assess the urgency of this support ticket on a scale of 1 (low) to 5 (critical).

Ticket content:
{state['ticket_content']}

Respond with only a single integer from 1 to 5."""
    
    response = model.invoke(prompt)
    urgency_str = response.content.strip()
    
    try:
        urgency = int(urgency_str)
        urgency = max(1, min(5, urgency))
    except ValueError:
        urgency = 3
    
    log_entry = f"Assessed urgency level: {urgency}/5"
    
    return {
        "urgency": urgency,
        "processing_log": [log_entry]
    }


def generate_response(state: TicketState) -> dict:
    """Generate response based on category and urgency."""
    model = init_chat_model("gpt-4o-mini", model_provider="openai")
    
    prompt = f"""Generate a professional support response for this ticket.

Ticket content: {state['ticket_content']}
Category: {state['category']}
Urgency: {state['urgency']}/5

Generate a helpful, professional response."""
    
    response = model.invoke(prompt)
    response_draft = response.content.strip()
    
    log_entry = f"Generated response draft (category: {state['category']}, urgency: {state['urgency']})"
    
    return {
        "response_draft": response_draft,
        "processing_log": [log_entry]
    }


def log_completion(state: TicketState) -> dict:
    """Add final log entry."""
    log_entry = f"Ticket processing completed. Category: {state['category']}, Urgency: {state['urgency']}, Response generated."
    
    return {
        "processing_log": [log_entry]
    }

In [22]:
# =============================================================================
# TODO 4: Build the Graph
# =============================================================================
# Linear flow: categorize -> assess -> generate -> log
#
# EXPERIMENT: Add conditional routing for urgent tickets
# =============================================================================

workflow = StateGraph(TicketState)

workflow.add_node("categorize", categorize_ticket)
workflow.add_node("assess", assess_urgency)
workflow.add_node("generate", generate_response)
workflow.add_node("log", log_completion)

workflow.add_edge(START, "categorize")
workflow.add_edge("categorize", "assess")
workflow.add_edge("assess", "generate")
workflow.add_edge("generate", "log")
workflow.add_edge("log", END)

agent = workflow.compile()

In [23]:
# =============================================================================
# Testing
# =============================================================================

test_tickets = [
    {
        "ticket_content": "Payment failed, charged twice! URGENT!",
        "customer_id": "CUST001",
        "category": "general",
        "urgency": 3,
        "response_draft": "",
        "processing_log": [],
        "tags": []
    },
    {
        "ticket_content": "How do I reset my password?",
        "customer_id": "CUST002",
        "category": "general",
        "urgency": 3,
        "response_draft": "",
        "processing_log": [],
        "tags": []
    },
    {
        "ticket_content": "Thanks for great service!",
        "customer_id": "CUST003",
        "category": "general",
        "urgency": 3,
        "response_draft": "",
        "processing_log": [],
        "tags": []
    }
]

print("Testing support ticket workflow:\n")
print("=" * 60)

for i, ticket in enumerate(test_tickets, 1):
    print(f"\nTest Ticket {i}: {ticket['ticket_content']}")
    print("-" * 60)
    
    result = agent.invoke(ticket)
    
    print(f"Category: {result['category']}")
    print(f"Urgency: {result['urgency']}/5")
    print(f"\nResponse Draft:\n{result['response_draft']}")
    print(f"\nProcessing Log ({len(result['processing_log'])} entries):")
    for log_entry in result['processing_log']:
        print(f"  - {log_entry}")
    print()

Testing support ticket workflow:


Test Ticket 1: Payment failed, charged twice! URGENT!
------------------------------------------------------------
Category: billing
Urgency: 5/5

Response Draft:
Subject: Urgent Assistance Regarding Double Charge

Dear [Customer's Name],

Thank you for reaching out to us regarding your billing issue. I understand how important this matter is, and I’m here to assist you.

I sincerely apologize for the inconvenience caused by the double charge on your account. To rectify this situation promptly, I will need to gather some additional information. Please provide me with the following details:

1. The email address associated with your account.
2. The date and amount of the transactions that were charged.
3. Any confirmation emails or transaction IDs related to these charges, if available.

Once I have this information, I will escalate your case to our billing department to investigate the issue and process any necessary refunds as quickly as possible. 

